In [383]:
import boto3
import pandas as pd
from pprint import pprint
import json

In [384]:
secrets = pd.read_json("../secrets/credentials.json")
creds = {"aws_access_key_id": secrets["aws"]["key_id"],
       "aws_secret_access_key": secrets["aws"]["key"]
}

client = boto3.client("kinesis",**creds)

stream_name = "finance"

In [385]:
stream = client.describe_stream(StreamName=stream_name)
pprint(stream)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '1431',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Mon, 29 Aug 2022 11:54:21 GMT',
                                      'x-amz-id-2': 'RJwbTPDtet8eYhAQ1hBebEZfs1Y1RCI5PEhYykUWd/AW6IZjNVRXjqkX0B3yTbdAQOVXRLTpmJhnkJ/dXiD9Sry/5XYAsWV0',
                                      'x-amzn-requestid': 'c3091255-d7cd-39ba-988f-b9780d2b2506'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'c3091255-d7cd-39ba-988f-b9780d2b2506',
                      'RetryAttempts': 0},
 'StreamDescription': {'EncryptionType': 'NONE',
                       'EnhancedMonitoring': [{'ShardLevelMetrics': []}],
                       'HasMoreShards': False,
                       'RetentionPeriodHours': 24,
                       'Shards': [{'HashKeyRange': {'EndingHashKey': '85070591730234615865843651857942052863',
                           

In [392]:
def get_shard_iterator(shard_id):
    shard_iterator = client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard_id,
        ShardIteratorType="TRIM_HORIZON"
    )
    return shard_iterator["ShardIterator"]
    
def get_records(shard_id):
    iterator = get_shard_iterator(shard_id)
    records = client.get_records(
        ShardIterator=iterator,
        Limit=5000
    )
    df = pd.DataFrame(records["Records"])
    return df

In [399]:
shards = ["000000000000", "000000000001", "000000000002"]
shard_data = map(lambda s: get_records(s), shards)

df = pd.concat(shard_data)
df.groupby("PartitionKey").count()

,SequenceNumber,ApproximateArrivalTimestamp,Data
PartitionKey,,,
12345,12,12,12
Conrad_rhm,2,2,2
RL_stocks,3,3,3
Uygar,1,1,1
attila_test,7,7,7
ceb,5,5,5
cinzia,333,333,333
johann_test,38,38,38
thau-test,7,7,7
